# OpenApiPy Jupyter Sample

In this Jupyter notebook we will use the Python package "ctrader-open-api" to get daily trend bars data from cTrader Open API.

Let's start.

If you haven't already installed the "ctrader-open-api" package, run the next code cell to install it via pip:

In [ ]:
!pip install ctrader-open-api

Then we have to import all necessary types:

In [ ]:
from ctrader_open_api import Client, Protobuf, TcpProtocol, Auth, EndPoints
from ctrader_open_api.messages.OpenApiCommonMessages_pb2 import *
from ctrader_open_api.messages.OpenApiCommonMessages_pb2 import *
from ctrader_open_api.messages.OpenApiMessages_pb2 import *
from ctrader_open_api.messages.OpenApiModelMessages_pb2 import *
from twisted.internet import reactor
import json

Now we use the "credentials-dev.json" file to get your Open API application credentials.
Be sure to populate it with your API application credentials and access token before running next cell:

In [ ]:
credentialsFile = open("credentials-dev.json")
credentials = json.load(credentialsFile)

Then we will create a client based our selected host type:

In [ ]:
host = EndPoints.PROTOBUF_LIVE_HOST if credentials["HostType"].lower() == "live" else EndPoints.PROTOBUF_DEMO_HOST
client = Client(host, EndPoints.PROTOBUF_PORT, TcpProtocol)

Now lets set the client callbacks:

In [ ]:
def onError(client, failure): # Call back for errors
    print("\nMessage Error: ", failure)

def connected(client): # Callback for client connection
    print("\nConnected")
    request = ProtoOAApplicationAuthReq()
    request.clientId = credentials["ClientId"]
    request.clientSecret = credentials["Secret"]
    deferred = client.send(request)
    deferred.addErrback(onError)

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected: ", reason)

def onMessageReceived(client, message): # Callback for receiving all messages
    print("\nMessage received: \n", Protobuf.extract(message))

# Setting optional client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

The last step is to run our client service, it will run inside Twisted reactor loop asynchronously:

In [ ]:
# Starting the client service
client.startService()
# Run Twisted reactor, we imported it earlier
reactor.run()